In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob

import tensorflow as tf

In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, GlobalMaxPooling2D
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [3]:
train_dir = '10_food_classes_10_percent/train/'
test_dir = '10_food_classes_10_percent/test/'

In [4]:
IMG_SIZE = (224, 224)
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(train_dir, label_mode='categorical',
                                                                    batch_size=32, image_size=IMG_SIZE)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(test_dir, label_mode='categorical',
                                                                   batch_size=32, image_size=IMG_SIZE)

Found 750 files belonging to 10 classes.


2022-06-22 23:33:28.592808: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 2500 files belonging to 10 classes.


In [7]:
data_augmentation = Sequential([
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomWidth(0.2),
    tf.keras.layers.RandomHeight(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomFlip('horizontal')
], name='data_augmentation')

In [8]:
checkpoint_path = 'model_checkpoints/min_val_loss'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                               save_best_only=True,
                                                               save_weights_only=True,
                                                               monitor='val_loss', mode='min')

In [9]:
import time
logdir = 'transfer_learning/{}'.format(time.time())
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [12]:
input_shape = (224, 224, 3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=input_shape, name='input_layer')
x = data_augmentation(inputs)
x = base_model(x, training=False)
x = GlobalMaxPooling2D(name='global_average_pooling_layer')(x)
outputs = Dense(10, activation='softmax', name='output_layer')(x)
model = tf.keras.Model(inputs, outputs)

model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(), metrics=['accuracy'])
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset, validation_steps=int(0.25*len(test_dataset)),
                    callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/10
24/24 [==============================] - 54s 2s/step - loss: 3.4756 - accuracy: 0.2427 - val_loss: 2.2178 - val_accuracy: 0.4424
Epoch 2/10
24/24 [==============================] - 47s 2s/step - loss: 1.8883 - accuracy: 0.4707 - val_loss: 1.5385 - val_accuracy: 0.5970
Epoch 3/10
24/24 [==============================] - 55s 2s/step - loss: 1.4053 - accuracy: 0.6093 - val_loss: 1.5122 - val_accuracy: 0.6135
Epoch 4/10
24/24 [==============================] - 52s 2s/step - loss: 1.2366 - accuracy: 0.6387 - val_loss: 1.0559 - val_accuracy: 0.7089
Epoch 5/10
24/24 [==============================] - 52s 2s/step - loss: 1.0606 - accuracy: 0.6947 - val_loss: 1.1017 - val_accuracy: 0.6974
Epoch 6/10
24/24 [==============================] - 51s 2s/step - loss: 0.8617 - accuracy: 0.7387 - val_loss: 1.0486 - val_accuracy: 0.7385
Epoch 7/10
24/24 [==============================] - 56s 2s/step - loss: 0.8633 - accuracy: 0.7320 - val_loss: 0.8963 - val_accuracy: 0.7336
Epoch 8/10
24/24 [==

In [14]:
model.evaluate(test_dataset)

79/79 [==============================] - 88s 1s/step - loss: 0.7693 - accuracy: 0.7868


[0.7692939043045044, 0.7868000268936157]

In [13]:
for idx, layer in enumerate(model.layers):
    print(idx, layer, layer.trainable)

0 <keras.engine.input_layer.InputLayer object at 0x7ff393f87df0> True
1 <keras.engine.sequential.Sequential object at 0x7ff3f4ca4f70> True
2 <keras.engine.functional.Functional object at 0x7ff395ebfa60> False
3 <keras.layers.pooling.global_max_pooling2d.GlobalMaxPooling2D object at 0x7ff394382a90> True
4 <keras.layers.core.dense.Dense object at 0x7ff3969dc1f0> True


### Finetune 30 last layer of EfficientNetB0

In [16]:
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False
    
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(0.0001), metrics=['accuracy'])
finetune_epochs = 10 + 10
finetune_history = model.fit(train_dataset, epochs=finetune_epochs, initial_epoch=history.epoch[-1],
                             validation_data=test_dataset, validation_steps=int(0.25*len(test_dataset)),
                             callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 10/20
24/24 [==============================] - 65s 2s/step - loss: 0.4720 - accuracy: 0.8493 - val_loss: 0.6433 - val_accuracy: 0.8322
Epoch 11/20
24/24 [==============================] - 60s 3s/step - loss: 0.3343 - accuracy: 0.8893 - val_loss: 0.6247 - val_accuracy: 0.8322
Epoch 12/20
24/24 [==============================] - 53s 2s/step - loss: 0.2774 - accuracy: 0.9000 - val_loss: 0.6903 - val_accuracy: 0.8322
Epoch 13/20
24/24 [==============================] - 57s 2s/step - loss: 0.1778 - accuracy: 0.9413 - val_loss: 0.6015 - val_accuracy: 0.8339
Epoch 14/20
24/24 [==============================] - 60s 3s/step - loss: 0.1638 - accuracy: 0.9520 - val_loss: 0.6605 - val_accuracy: 0.8273
Epoch 15/20
24/24 [==============================] - 62s 3s/step - loss: 0.1135 - accuracy: 0.9640 - val_loss: 0.5959 - val_accuracy: 0.8405
Epoch 16/20
24/24 [==============================] - 52s 2s/step - loss: 0.1071 - accuracy: 0.9653 - val_loss: 0.6080 - val_accuracy: 0.8388
Epoch 17/20
2

In [17]:
model.evaluate(test_dataset)

79/79 [==============================] - 93s 1s/step - loss: 0.7307 - accuracy: 0.8296


[0.730686604976654, 0.8295999765396118]

In [18]:
for idx, layer in enumerate(model.layers):
    print(idx, layer, layer.trainable)

0 <keras.engine.input_layer.InputLayer object at 0x7ff393f87df0> True
1 <keras.engine.sequential.Sequential object at 0x7ff3f4ca4f70> True
2 <keras.engine.functional.Functional object at 0x7ff395ebfa60> True
3 <keras.layers.pooling.global_max_pooling2d.GlobalMaxPooling2D object at 0x7ff394382a90> True
4 <keras.layers.core.dense.Dense object at 0x7ff3969dc1f0> True


In [19]:
test_dataset.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [22]:
sample = test_dataset.take(1)

In [25]:
for i in sample:
    print(i)

(<tf.Tensor: shape=(32, 224, 224, 3), dtype=float32, numpy=
array([[[[2.00729599e+02, 2.29158173e+02, 2.21801025e+02],
         [1.99209183e+02, 2.25311218e+02, 2.16260208e+02],
         [2.04204086e+02, 2.24352036e+02, 2.12841827e+02],
         ...,
         [2.17193985e+01, 1.59336939e+01, 2.03622818e+01],
         [2.65000687e+01, 2.15000687e+01, 2.75000687e+01],
         [2.42704220e+01, 1.92704220e+01, 2.52704220e+01]],

        [[1.99295914e+02, 2.14724487e+02, 2.10918365e+02],
         [2.05331635e+02, 2.16489807e+02, 2.10336731e+02],
         [1.92806122e+02, 1.97306122e+02, 1.86862244e+02],
         ...,
         [2.70305882e+01, 2.12448826e+01, 2.56734715e+01],
         [2.58469582e+01, 2.08469582e+01, 2.68469582e+01],
         [2.58826542e+01, 2.08826542e+01, 2.68826542e+01]],

        [[2.13285721e+02, 2.19066330e+02, 2.13862244e+02],
         [2.22709183e+02, 2.23678574e+02, 2.14265305e+02],
         [1.82000015e+02, 1.75352051e+02, 1.57566330e+02],
         ...,
         